In [57]:
# This script should take the em_centroids h5_file and query the agglo_id
import numpy as np
import pandas as pd
import h5py

from brainmaps_api_fcn.equivalence_requests import EquivalenceRequests
from brainmaps_api_fcn.subvolume_requests import SubvolumeRequest

from google.cloud import bigquery

import os

from tqdm import tqdm

from scripts.config_model import save_experiment_config, tree
from scripts.sample_db import SampleDB
from scripts.utils.image_utils import load_tiff_as_hyperstack, save_array_as_hyperstack_tiff

from google.auth import exceptions

import time

In [61]:

# Get the agglomerated ID that contains EM centroid
def get_agglo_group_from_point(sa, volume_id, stack_change, centroid_xyz):
    sr = SubvolumeRequest(sa, volume_id)
    vol = sr.get_subvolume(centroid_xyz, size= [1,1,1], change_stack_id = stack_change)
    agglo_id  =  int(np.unique(vol[vol>0])[0])
        #
        # print(f"ID found: {agglo_id}")

    er = EquivalenceRequests(sa, volume_id, stack_change)
    group = er.get_groups(agglo_id)
    return group


def get_fresh_client():
    return bigquery.Client(project="aggloproofreading")


def get_agglo_group_with_retry(sa, volume_id, stack_change, centroid_xyz, max_retries=5):
    for attempt in range(max_retries):
        try:
            sr = SubvolumeRequest(sa, volume_id)
            vol = sr.get_subvolume(centroid_xyz, size=[1,1,1], change_stack_id=stack_change)
            agglo_id = int(np.unique(vol[vol>0])[0])
            
            er = EquivalenceRequests(sa, volume_id, stack_change)
            group = er.get_groups(agglo_id)
            return group
        except exceptions.RefreshError:
            if attempt == max_retries - 1:
                raise
            print(f"Try {attempt+1}/{max_retries}")
            time.sleep(attempt)

In [36]:
client = bigquery.Client(project="aggloproofreading")

# Step 1: Load the sample database
db_path = r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\sample_db.csv'
sample_db = SampleDB()
sample_db.load(db_path)
# Step 2: Load experiment configuration
sample_id = '20220426_RM0008_130hpf_fP1_f3'
exp = sample_db.get_sample(sample_id)

# TODO: add paths to sample
# Input: volume data
sa = r"\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\Reconstruction\fmi-friedrich-4dd3f21e665d.json"
volume_id = r"280984173682:montano_rm2_ngff:raw_230701_seg_240316fb"
stack_change = "240705d_rsg9_spl"

In [37]:
em_centroids_path = os.path.join(exp.paths.em_path, f"{exp.sample.id}_em_centroids.h5")


In [38]:
# Read centroids from HDF5 file and do transformation to neuroglancer
with h5py.File(em_centroids_path, 'r') as f:
    # Get centroids coordinates
    data = f['centroids/coordinates'][:]
    centroids = data[:,1:]
    
    # Get transformation parameters
    transformation_matrix = f['metadata/transformations/bigwarp2neuroglancer/transformation_matrix'][:]
    rotated_cropped_stack_center_shift = f['metadata/transformations/bigwarp2neuroglancer/rotated_cropped_stack_center_shift'][:]
    rotated_stack_center_shift = f['metadata/transformations/bigwarp2neuroglancer/rotated_stack_center_shift'][:]
    cropped_shift = f['metadata/transformations/bigwarp2neuroglancer/cropped_shift'][:]
    ng_shift = f['metadata/transformations/bigwarp2neuroglancer/ng_shift'][:]
    downsampled_factor = f['metadata/transformations/bigwarp2neuroglancer/downsampled_factor'][:]
    zyx2xyz = f['metadata/transformations/bigwarp2neuroglancer'].attrs['zyx2xyz']
    # Apply transformations to get Neuroglancer coordinates
    # 1. Center points around origin
    centered_points = centroids - rotated_cropped_stack_center_shift
    
    # 2. Apply transformation matrix
    transformed_centered = np.dot(centered_points, transformation_matrix.T)
    
    # 3. Move to target space and apply shifts
    transformed_points = (transformed_centered + 
                         rotated_stack_center_shift + 
                         cropped_shift)
    if f['metadata/transformations/bigwarp2neuroglancer/downsampled_factor']:
        downsampled_factor = f['metadata/transformations/bigwarp2neuroglancer/downsampled_factor'][:]
        transformed_points = transformed_points*downsampled_factor
    
    # 4. Convert from ZYX to XYZ if needed
    if f['metadata/transformations/bigwarp2neuroglancer'].attrs['zyx2xyz']:
        transformed_points = transformed_points[:, ::-1]
    
    # 5. Correct for shift in neuroglancer
    transformed_points = transformed_points + ng_shift



In [ ]:
lut_path = os.path.join(exp.paths.clem_path, f"{exp.sample.id}_lut.h5")
error_neurons = []
with h5py.File(lut_path, 'w') as hdf5_file:
    # Create metadata group with source information
    metadata_group = hdf5_file.create_group('metadata')
    metadata_group.attrs['source_stack'] = os.path.join(exp.paths.em_path, '20220426_RM0008_130hpf_fP1_f3_fine_aligned_downsampled_16_em_stack_cropped_woResin_rough_rotated_to_LM.tif')
    metadata_group.attrs['source_mask'] = os.path.join(exp.paths.em_path, '20220426_RM0008_130hpf_fP1_f3_fine_aligned_downsampled_16_em_stack_cropped_woResin_rough_rotated_to_LM_mask_filtered.tif')
    metadata_group.attrs['sample_id'] = exp.sample.id
    metadata_group.attrs['volume_id'] = volume_id
    metadata_group.attrs['stack_change'] = stack_change
    
    # Store transformation parameters
    transformations_group = metadata_group.create_group('transformations')
    ng2bw_group = transformations_group.create_group('neuroglancer2bigwarp')
    ng2bw_group.create_dataset('transformation_matrix', data=transformation_matrix)
    ng2bw_group.create_dataset('rotated_cropped_stack_center_shift', data=rotated_cropped_stack_center_shift)
    ng2bw_group.create_dataset('rotated_stack_center_shift', data=rotated_stack_center_shift)
    ng2bw_group.create_dataset('cropped_shift', data=cropped_shift)
    ng2bw_group.create_dataset('ng_shift', data=ng_shift)
    ng2bw_group.attrs['zyx2xyz'] = zyx2xyz
    
    # Create neurons group
    neurons_group = hdf5_file.create_group('neurons')
    
    # Add data for each neuron
    # Step 2: Query and update segment information
    for neuron_id in tqdm(range(len(centroids)), desc="Finding agglomeration ids and their segments"):
        
        neuron_group = neurons_group.create_group(f'neuron_{neuron_id}')
        neuron_group.create_dataset('em_centroid_bw', data=centroids[neuron_id])
        neuron_group.create_dataset('em_centroid_ng', data=transformed_points[neuron_id])
        
        segments_group = neuron_group.create_group('segments')
        
        try:
            # Get agglomeration ID and segments
            agglo_group = get_agglo_group_with_retry(sa, volume_id, stack_change, 
                                                    np.round(transformed_points[neuron_id]).astype(int))
            neuron_group.attrs['agglo_id'] = list(agglo_group.keys())[0]
            
            for agglo_id, segment_ids in agglo_group.items():
   
                # Construct SQL query for the current batch
                query = f"""
                SELECT id AS segment_id,
                       bbox.start.x AS start_x,
                       bbox.start.y AS start_y,
                       bbox.start.z AS start_z,
                       bbox.size.x AS size_x,
                       bbox.size.y AS size_y,
                       bbox.size.z AS size_z,
                       num_voxels  
                FROM `fmi-friedrich.ruth_ob.raw_230701_seg_240316fb_objinfo` 
                WHERE id IN {tuple(segment_ids)}
                """

                # Execute the query
                query_job = client.query(query)

                # Store results in HDF5
                for row in query_job.result():
                    segment_group = segments_group.create_group(str(row['segment_id']))
                    segment_group.create_dataset('bbox_start', data=[row['start_x'], row['start_y'], row['start_z']])
                    segment_group.create_dataset('bbox_size', data=[row['size_x'], row['size_y'], row['size_z']])
                    segment_group.create_dataset('num_voxels', data=row['num_voxels'])
                    segment_group.attrs['axes_order'] = "xyz"

    
        except Exception as e:
            error_neurons.append(neuron_id)
            print(f"Error processing neuron {neuron_id}: {e}")




Finding agglomeration ids and their segments:   0%|          | 0/9462 [00:00<?, ?it/s]

In [51]:
lut_path = os.path.join(exp.paths.clem_path, f"{exp.sample.id}_lut.h5")
error_neurons = []
with h5py.File(lut_path, 'w') as hdf5_file:
    # Create metadata group with source information
    metadata_group = hdf5_file.create_group('metadata')
    metadata_group.attrs['source_stack'] = os.path.join(exp.paths.em_path, '20220426_RM0008_130hpf_fP1_f3_fine_aligned_downsampled_16_em_stack_cropped_woResin_rough_rotated_to_LM.tif')
    metadata_group.attrs['source_mask'] = os.path.join(exp.paths.em_path, '20220426_RM0008_130hpf_fP1_f3_fine_aligned_downsampled_16_em_stack_cropped_woResin_rough_rotated_to_LM_mask_filtered.tif')
    metadata_group.attrs['sample_id'] = exp.sample.id
    metadata_group.attrs['volume_id'] = volume_id
    metadata_group.attrs['stack_change'] = stack_change
    
    # Store transformation parameters
    transformations_group = metadata_group.create_group('transformations')
    ng2bw_group = transformations_group.create_group('neuroglancer2bigwarp')
    ng2bw_group.create_dataset('transformation_matrix', data=transformation_matrix)
    ng2bw_group.create_dataset('rotated_cropped_stack_center_shift', data=rotated_cropped_stack_center_shift)
    ng2bw_group.create_dataset('rotated_stack_center_shift', data=rotated_stack_center_shift)
    ng2bw_group.create_dataset('cropped_shift', data=cropped_shift)
    ng2bw_group.create_dataset('ng_shift', data=ng_shift)
    ng2bw_group.attrs['zyx2xyz'] = zyx2xyz
    
    # Create neurons group
    neurons_group = hdf5_file.create_group('neurons')
    
    # Add data for each neuron
    # Step 2: Query and update segment information
    for neuron_id in tqdm(range(len(centroids)), desc="Finding agglomeration ids and their segments"):
        neuron_group = neurons_group.create_group(f'neuron_{neuron_id}')
        neuron_group.create_dataset('em_centroid_bw', data=centroids[neuron_id])
        neuron_group.create_dataset('em_centroid_ng', data=transformed_points[neuron_id])
        
        segments_group = neuron_group.create_group('segments')
        
        try:
            # Get agglomeration ID and segments
            agglo_group = get_agglo_group_from_point(sa, volume_id, stack_change, 
                                                    np.round(transformed_points[neuron_id]).astype(int))
            neuron_group.attrs['agglo_id'] = list(agglo_group.keys())[0]
            
            # Get segment IDs from agglo_group
            for agglo_id, segment_ids in agglo_group.items():
   
                # Construct SQL query for the current batch
                query = f"""
                SELECT id AS segment_id,
                       bbox.start.x AS start_x,
                       bbox.start.y AS start_y,
                       bbox.start.z AS start_z,
                       bbox.size.x AS size_x,
                       bbox.size.y AS size_y,
                       bbox.size.z AS size_z,
                       num_voxels  
                FROM `fmi-friedrich.ruth_ob.raw_230701_seg_240316fb_objinfo` 
                WHERE id IN {tuple(segment_ids)}
                """

                # Execute the query
                query_job = client.query(query)

                # Store results in HDF5
                for row in query_job.result():
                    segment_group = segments_group.create_group(str(row['segment_id']))
                    segment_group.create_dataset('bbox_start', data=[row['start_x'], row['start_y'], row['start_z']])
                    segment_group.create_dataset('bbox_size', data=[row['size_x'], row['size_y'], row['size_z']])
                    segment_group.create_dataset('num_voxels', data=row['num_voxels'])
                    segment_group.attrs['axes_order'] = "xyz"
    
        except Exception as e:
            error_neurons.append(neuron_id)
            print(f"Error processing neuron {neuron_id}: {e}")




Finding agglomeration ids and their segments:   0%|          | 13/9462 [01:02<9:47:02,  3.73s/it] 

Error processing neuron 12: ('invalid_grant: Invalid JWT Signature.', {'error': 'invalid_grant', 'error_description': 'Invalid JWT Signature.'})


Finding agglomeration ids and their segments:   0%|          | 23/9462 [01:52<11:48:39,  4.50s/it]

Error processing neuron 22: ('invalid_grant: Invalid JWT Signature.', {'error': 'invalid_grant', 'error_description': 'Invalid JWT Signature.'})


Finding agglomeration ids and their segments:   0%|          | 31/9462 [02:28<10:30:40,  4.01s/it]

...

Finding agglomeration ids and their segments:   0%|          | 46/9462 [03:19<5:47:54,  2.22s/it]

Error processing neuron 45: ('invalid_grant: Invalid JWT Signature.', {'error': 'invalid_grant', 'error_description': 'Invalid JWT Signature.'})


Finding agglomeration ids and their segments:   1%|          | 48/9462 [03:24<5:52:58,  2.25s/it]

Error processing neuron 47: ('invalid_grant: Invalid JWT Signature.', {'error': 'invalid_grant', 'error_description': 'Invalid JWT Signature.'})


Finding agglomeration ids and their segments:   1%|          | 48/9462 [03:26<11:16:06,  4.31s/it]

KeyboardInterrupt



In [ ]:
# Filter only dendrites
# Filter only OB
# Create convex hulls 

